In [23]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo cp chi_tra.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!rm fig_*.jpg

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

try:
    from PIL import Image
except ImportError:
    import Image

import pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [25]:
mpl.rcParams['image.cmap'] = 'gray'

# Read the image
img = cv2.imread("img.jpg", 0)
#img = cv2.imread("img-b.jpg", 0)
#img = cv2.imread("img0.2cm.jpg", 0)
#img = cv2.imread("img-tpe.jpg", 0)

 # Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY|     cv2.THRESH_OTSU)
# Invert the image
img_bin = 255-img_bin 
cv2.imwrite("Image_bin.jpg",img_bin)
#plt.axis('off')
#plt.imshow(img_bin);

True

In [30]:
x = 70
global horizontal_lines_img, verticle_lines_img, kernel
kernel_length = np.array(img).shape[1]//x

# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
# A kernel of (3 X 3) ones.
#kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
cv2.imwrite("Image_verticle_lines.jpg",verticle_lines_img)
# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
cv2.imwrite("Image_horizontal_lines.jpg",horizontal_lines_img)

# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha
# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#plt.axis('off')
#plt.imshow(img_final_bin)
cv2.imwrite("Image_bin_final.jpg",img_final_bin)



True

In [0]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:(b[1][1],b[1][0]), reverse=False))
        #key=lambda b:b[1][i], reverse=False))

    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)





In [64]:
# Find contours for image, which will detect all the boxes
contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Sort all the contours by top to bottom.
(contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

idx = 0
for c in contours:
    # Returns the location and width,height for every contour
    x, y, w, h = cv2.boundingRect(c)
# If the box height is greater then 20, widht is >80, then only save it as a box in "cropped/" folder.
    #if (w > 80 and h > 20) and w > 3*h:
    if (w > 80 and h > 20) and w > 1.5*h:
        idx += 1
        new_img = img[y:y+h, x:x+w]
        cv2.imwrite('fig_'+str(idx)+'.jpg', new_img)
        #defines the number of cells in the outcome
        #plt.subplot(100,1, idx)
        #plt.imshow(new_img)
        #plt.axis('off')
        
        #print('== fig_'+str(idx)+'.jpg ==')
        #img2 = Image.open('fig_'+str(idx)+'.jpg')
        text = pytesseract.image_to_string(new_img, lang='chi_tra')  
        #英文 'eng'、簡體中文 'chi_sim'、繁體中文 'chi_tra'
        print('fig_'+str(idx)+'.jpg ==> ' + text)
    else:
        print ("x,y,w,h = ",x,y,w,h)

x,y,w,h =  0 0 1239 1754
x,y,w,h =  72 207 1115 1080
fig_1.jpg ==> 
fig_2.jpg ==> 
fig_3.jpg ==> 
fig_4.jpg ==> 大 智 國 小
fig_5.jpg ==> 08BEACOA0412
fig_6.jpg ==> 台 中 市 立 大 智 國 小
fig_7.jpg ==> 40145 臺 中 市 東 區 大 智 路 359 號
fig_8.jpg ==> 樂 業 國 小
fig_9.jpg ==> 08BEACOA0404
fig_10.jpg ==> 台 中 市 立 樂 業 國 小
fig_11.jpg ==> 40152 臺 中 市 東 區 樂 業 路 60 號


KeyboardInterrupt: ignored

In [66]:
img2 = Image.open('fig_1.jpg')
#text = pytesseract.image_to_string(img2)  
text = pytesseract.image_to_string(img2, lang='chi_tra')  
print(text)
img2 = Image.open('fig_2.jpg')
text = pytesseract.image_to_string(img2)  
print(text)
img2 = Image.open('fig_3.jpg')
text = pytesseract.image_to_string(img2)  
print(text)




aA
